<a href="https://colab.research.google.com/github/parham-nasiri/App/blob/main/Gpu_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
import pandas as pd
import json
import torch
import torch.nn as nn
import numpy as np
from torch.utils.data import Dataset
from sklearn.preprocessing import MinMaxScaler
from torch.optim import Adam
from torch.utils.data import DataLoader, TensorDataset
from google.colab import drive

In [27]:
from google.colab import files
drive.mount('/content/drive')
file_path = '/content/drive/MyDrive/gpu_data.txt'

#uploaded = files.upload()

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [29]:
with open(file_path, "r") as f:
#with open("gpu_data.txt", "r") as f:
  data = json.load(f)

In [31]:
gpu_data = {}
for item in data["data"]["result"]:
    gpu_name = item["metric"]["gpu"]
    values = item["values"]

    for timestamp, value in values:
        if timestamp not in gpu_data:
            gpu_data[timestamp] = {}
        gpu_data[timestamp][gpu_name] = float(value)


sorted_times = sorted(gpu_data.keys())
all_gpus = sorted({gpu for t in gpu_data.values()for gpu in t.keys()})

final_vectors = []

for t in sorted_times:
    vector = [gpu_data[t].get(gpu, 0) for gpu in all_gpus]
    final_vectors.append(vector)

tensor_data = torch.tensor(final_vectors, dtype=torch.float32)

In [32]:
scaler = MinMaxScaler()
scaled_data = scaler.fit_transform(tensor_data)
scaled_data = torch.tensor(scaled_data, dtype=torch.float32)

In [33]:
def create_sequence(data,seq_lenght):
  X,y = [] ,[]
  for i in range(len(data)-seq_lenght):
    X.append(data[i:i+seq_lenght])
    y.append(data[i+seq_lenght])
  return torch.stack(X),torch.stack(y)
seq_lenght = 20
X,y = create_sequence(scaled_data,seq_lenght)

In [17]:
train_size = int(0.8*len(X))
X_train= X[:train_size]
y_train= y[:train_size]
X_test= X[:train_size]
y_test= y[:train_size]


In [18]:
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

/tmp/ipython-input-1211342857.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(X, dtype=torch.float32)
/tmp/ipython-input-1211342857.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y = torch.tensor(y, dtype=torch.float32)


In [19]:
class GPUDataset(Dataset):
  def __init__(self,X,y):
    self.X = X
    self.y = y
  def __len__(self):
    return len(self.X)
  def __getitem__(self,idx):
    return self.X[idx],self.y[idx]

In [20]:
batch_size = 32

train_loader = DataLoader(GPUDataset(X_train, y_train),
                          batch_size=batch_size,
                          shuffle=False)

test_loader = DataLoader(GPUDataset(X_test, y_test),
                         batch_size=batch_size,
                         shuffle=False)

In [21]:
class LSTMModel(nn.Module):
  def __init__(self,input_size,hidden_size,num_layers,num_size,dropout):
    super().__init__()
    self.lstm = nn.LSTM(
        input_size=input_size,
        hidden_size=hidden_size,
        num_layers=num_layers,
        batch_first=True,
        dropout=dropout
    )
    self.fc = nn.Linear(hidden_size,num_size)
  def forward(self,X):
    out, _ = self.lstm(X)
    out = out[:, -1, :]
    out = self.fc(out)
    return out

In [22]:
device = torch.device("cuda" if torch.cuda.is_available()else "cpu")
model = LSTMModel(input_size= tensor_data.shape[1],hidden_size= 64,num_size=tensor_data.shape[1],num_layers= 2,dropout= 0.2).to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

In [23]:
print(X_train.shape)
print(y_train.shape)

torch.Size([6895, 20, 8])
torch.Size([6895, 8])


In [24]:
epochs = 50

for epoch in range(epochs):
    model.train()
    train_loss = 0

    for X_batch, y_batch in train_loader:
        X_batch = X_batch.to(device)
        y_batch = y_batch.to(device)

        optimizer.zero_grad()
        output = model(X_batch)
        loss = criterion(output, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)

    if epoch % 5 == 0:
        print(f"Epoch {epoch}, Train Loss: {train_loss:.6f}")

Epoch 0, Train Loss: 0.047431
Epoch 5, Train Loss: 0.019088
Epoch 10, Train Loss: 0.008769
Epoch 15, Train Loss: 0.004310
Epoch 20, Train Loss: 0.003460
Epoch 25, Train Loss: 0.003154
Epoch 30, Train Loss: 0.002934
Epoch 35, Train Loss: 0.002761
Epoch 40, Train Loss: 0.002747
Epoch 45, Train Loss: 0.002639


In [34]:
torch.save(model.state_dict(), "gpu_lstm_model.pth")